<a href="https://github.com/gutris1/segsmaker">
  <img alt="GitHub repo" src="https://img.shields.io/badge/GitHub-6e5494?style=for-the-badge&logo=github&logoColor=white"/>
</a><br>

*   get your civitai api key from [here](https://civitai.com/user/account)

In [ ]:
# @title <b><font color='orange'>WebUI Installer</font></b> {"display-mode":"form"}

Webui = 'ReForge' # @param ["A1111", "Forge", "ReForge", "Forge-Classic", "Forge-Neo", "ComfyUI", "SwarmUI"]
Civitai___Key = '' # @param { type: "string", placeholder: "Your Civitai API Key (required)" }
HF_Read_Token = '' # @param { type: "string", placeholder: "Your Huggingface READ Token (optional)" }
Mount__GDrive = 'No' # @param ["Yes", "No"]

mount = Mount__GDrive

if mount == 'Yes':
    from google.colab import drive
    drive.mount('/content/drive')

!curl -sLo /content/setup.py https://github.com/gutris1/segsmaker/raw/main/script/KC/setup.py
%run /content/setup.py --webui="$Webui" --civitai_key="$Civitai___Key" --hf_read_token="$HF_Read_Token"

if mount == 'Yes':
    from pathlib import Path

    d = Path('/content/drive/MyDrive/Segsmaker')

    for n, p in {'checkpoint': CKPT, 'lora': LORA, 'vae': VAE, 'embeddings': Embeddings}.items():
        f = d / n
        f.mkdir(parents=True, exist_ok=True)
        s = p / f'drive-{n}'
        s.symlink_to(f, target_is_directory=True)

    !rm -rf $WebUI_Output
    o = d / {'ComfyUI': 'comfyui-output', 'SwarmUI': 'swarmui-output'}.get(Webui, 'output')
    o.mkdir(parents=True, exist_ok=True)
    WebUI_Output.symlink_to(o, target_is_directory=True)

    if Webui not in {'ComfyUI', 'SwarmUI'}:
        wc = WebUI / 'cache'
        !rm -rf $wc
        c = d / 'cache'
        c.mkdir(parents=True, exist_ok=True)
        wc.symlink_to(c, target_is_directory=True)

In [ ]:
''' These are placeholders for commands with empty inputs. '''
''' Replace them with actual values or skip/delete this cell. '''


''' SD Extensions / ComfyUI Custom Nodes '''
%cd -q $Extensions
!git clone

''' VAE '''
%cd -q $VAE
%download

''' Embeddings '''
%cd -q $Embeddings
%download

''' Upscalers '''
%cd -q $Upscalers
%download

''' FLUX Unet '''
%cd -q $UNET
%download

''' FLUX Clip '''
%cd -q $CLIP
%download

In [ ]:
''' Checkpoint '''
%cd -q $CKPT
%download https://huggingface.co/pantat88/back_up/resolve/main/bigblu25dmix25DStyle_v10.safetensors

''' LoRA '''
%cd -q $LORA
%download https://civitai.com/models/122359/detail-tweaker-xl
%download https://civitai.com/models/669571/pony-add-more-details details-add-more-pony.safetensors

In [ ]:
''' Controlnet '''
%run $Controlnet_Widget

## Launcher
args list :
-  **A1111** = `--xformers`
- **Forge** = `--disable-xformers --opt-sdp-attention --cuda-stream`
- **ReForge** = `--xformers --cuda-stream`
- **Forge-Classic** = `--xformers --cuda-stream --persistent-patches`
- **Forge-Neo** = `--xformers --cuda-stream`
- **ComfyUI** = `--dont-print-server --use-pytorch-cross-attention`
- **SwarmUI** = `--launch_mode none`
<br><br>

For ComfyUI, add `--skip-comfyui-check` to skip checking the main requirements and custom node dependencies

Add **--N=ngrok_token** to start NGROK tunnel<br>
Add **--Z=zrok_token** to start ZROK tunnel

In [ ]:
%cd -q $WebUI
%run segsmaker.py --xformers